## Kubernetes Clusterwide SSL Demo

### Setup

Create a Kind cluster by using an ansible playbook from the project `ansible` folder.


In [ ]:
!cd ../ansible && ansible-playbook playbooks/kind-cluster.yaml

In [ ]:
!cd ../ansible && ansible-playbook playbooks/setup-ecosystem.yaml

In [25]:
!helm upgrade --install seldon-core-v2-crds  ../k8s/helm-charts/seldon-core-v2-crds -n seldon-mesh

Release "seldon-core-v2-crds" does not exist. Installing it now.
NAME: seldon-core-v2-crds
LAST DEPLOYED: Sat May 13 10:40:02 2023
NAMESPACE: seldon-mesh
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [ ]:
!kubectl create namespace ns1

In [ ]:
!kubectl create namespace ns2

In [ ]:
!cat ../k8s/samples/values-tls-dataplane-controlplane-example.yaml

In [26]:
!helm install seldon-v2 ../k8s/helm-charts/seldon-core-v2-setup/ -n seldon-mesh --set controller.clusterwide=true --values ../k8s/samples/values-tls-dataplane-controlplane-example.yaml

NAME: seldon-v2
LAST DEPLOYED: Sat May 13 10:48:56 2023
NAMESPACE: seldon-mesh
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [ ]:
!helm install seldon-v2-certs ../k8s/helm-charts/seldon-core-v2-certs/ -n ns1

In [ ]:
!helm install seldon-v2-certs ../k8s/helm-charts/seldon-core-v2-certs/ -n ns2

In [27]:
!helm install seldon-v2-runtime ../k8s/helm-charts/seldon-core-v2-runtime  -n ns1

NAME: seldon-v2-runtime
LAST DEPLOYED: Sat May 13 10:49:22 2023
NAMESPACE: ns1
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [ ]:
!helm install seldon-v2-runtime ../k8s/helm-charts/seldon-core-v2-runtime  -n ns2

### Setup TLS Config

In [8]:
MESH_IP=!kubectl get svc seldon-mesh -n ns1 -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP_NS1=MESH_IP[0]
import os
os.environ['MESH_IP_NS1'] = MESH_IP_NS1
MESH_IP_NS1

'172.21.255.2'

In [ ]:
MESH_IP=!kubectl get svc seldon-mesh -n ns2 -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP_NS2=MESH_IP[0]
import os
os.environ['MESH_IP_NS2'] = MESH_IP_NS2
MESH_IP_NS2

In [ ]:
!seldon config add tls ${PWD}/config-dataplane-tls.json

In [ ]:
!seldon config activate tls

### Launch model in namespace ns1

In [9]:
!kubectl create -f ./models/sklearn-iris-gs.yaml -n ns1

model.mlops.seldon.io/iris created


In [10]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ns1

model.mlops.seldon.io/iris condition met


In [11]:
!seldon model infer iris --inference-host ${MESH_IP_NS1}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "b0629eed-5438-4dfc-8314-db461cdb08a0",
	"parameters": {},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1,
				1
			],
			"datatype": "INT64",
			"data": [
				2
			]
		}
	]
}


### Launch model in namespace ns2

In [ ]:
!kubectl create -f ./models/sklearn-iris-gs.yaml -n ns2

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ns2

In [ ]:
!seldon model infer iris --inference-host ${MESH_IP_NS2}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

## TearDown

In [17]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml -n ns1

Error from server (NotFound): error when deleting "./models/sklearn-iris-gs.yaml": models.mlops.seldon.io "iris" not found


In [ ]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml -n ns2

In [22]:
!helm delete seldon-v2-runtime -n ns1

release "seldon-v2-runtime" uninstalled


In [23]:
!helm delete seldon-v2-runtime -n ns2

Error: uninstall: Release not loaded: seldon-v2-runtime: release: not found


In [24]:
!helm delete seldon-v2  -n seldon-mesh

release "seldon-v2" uninstalled
